# Calibration

This file will describe sources for the data in this repository, as well as any extra processing information.

## Emissions

The processes for the emissions data can be found in the [RFF Socioeconomic Projections repository](https://github.com/rffscghg/rff-socioeconomic-projections).  Outputs from this repository are copied into _data/emissions_.

## Socioeconomic

The processes for the socioeconomic data can be found in the [RFF Socioeconomic Projections repository](https://github.com/rffscghg/rff-socioeconomic-projections).  Outputs from this repository are copied into _data/socioeconomic_.

## Baseline Mortality

In order to run some damage functions, users need baseline mortality data harmonized to a given population trajectory.  Baseline mortality data found in _data/mortality_ was derived from the _death_rates.csv_ provided to the RFF team on October 7th from Hana Sevcikova and available (privately) here: https://drive.google.com/open?id=1TCYgzRJyt-8wadRcCfxDdpqF84t6oZfj&authuser=hanas%40uw.edu&usp=drive_fs. Column `DeathRate` is the annual deaths per 1000 people. `PopAvg` is the denominator (average between two time periods) and `PopStart` is population at the start of the time interval. Values are average deaths per 1000 people. 

There is one mortality death rate trajectory for each population trajectory, and each RFF SP is matched to one of these 1000 trajectories. Post processing of _death_rates.csv_ into _TempMortality_cdr_rf_sp_TrajectoryX.csv_ is below.

In [ ]:
using DataFrames, Query, CSVFiles

codes = load(joinpath(@__DIR__, "..", "data", "keys", "NumericCodes_to_ISO3.csv")) |> DataFrame 

# TODO upload this file in pieces, or upload to Zotero etc.
df = load("death_rates.csv") |> 
    DataFrame |> 
    @filter(_.LocID in codes.NumericCode) |>
    @select(:LocID, :Year, :Trajectory, :DeathRate) |> 
    DataFrame

# get ISO3 codes
idxs = indexin(df.LocID, codes.NumericCode)
insertcols!(df, 1, :ISO3 => codes.ISO3[idxs] 
select!(df, Not(:LocID))